# Multi-Agent with Tools Example

This notebook demonstrates a multi-agent setup using the OpenAI Agent SDK, where each agent is equipped with specific tools to perform its tasks. We will create a 'researcher' agent that can search the web and a 'writer' agent that can write content to a file.

In [1]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool
from agents.extensions.models.litellm_model import LitellmModel

from dotenv import load_dotenv
import os
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

BASE_URL = os.getenv("EXAMPLE_BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/"
API_KEY = os.getenv("EXAMPLE_API_KEY") or gemini_api_key
MODEL_NAME = os.getenv("EXAMPLE_MODEL_NAME") or "gemini-1.5-pro"
import nest_asyncio #this is needed to run async code in Jupyter notebooks
nest_asyncio.apply()
client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)
set_tracing_disabled(disabled=True)

## The Agent Loop
The agent loop is the core of the agent's operation. It's a continuous cycle where the agent:
1. **Receives a prompt:** This can be from a user or another agent.
2. **Thinks:** The agent's underlying model processes the prompt and decides on the best course of action. This might involve using a tool, handing off to another agent, or responding directly.
3. **Acts:** The agent executes the chosen action. This could be calling a tool's function, passing the task to a sub-agent, or generating a response.
4. **Observes:** The agent receives the result of its action (e.g., the output of a tool, the response from a sub-agent).
5. **Repeats:** The agent goes back to the 'Thinks' step, incorporating the new information to decide on the next action. This loop continues until the task is complete.

### Sub-Agent 1: The Researcher

In [3]:
from duckduckgo_search import DDGS

@function_tool
def web_search(query: str) -> str:
    """Searches the web for the given query and returns the results."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=3)]
    return str(results)
researcher_agent = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
    name="researcher_agent",
    instructions="You are an expert researcher. Use the web_search tool to find information on the given topic.",
    tools=[web_search],
    handoff_description="Use this agent to conduct research and find information on a specific topic."
)

### Sub-Agent 2: The Writer

In [4]:
@function_tool
def write_file(file_path: str, content: str) -> str:
    """Writes the given content to a file at the specified path."""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"Successfully wrote to {file_path}."

writer_agent = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
    name="writer_agent",
    instructions="You are an expert writer. Use the write_file tool to write the given content to a file.",
    tools=[write_file],
    handoff_description="Use this agent to write content to a file."
)

### The Supervisor Agent

In [5]:
supervisor_agent = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
    name="supervisor",
    instructions="You are the supervisor. Your job is to coordinate the researcher and writer agents to complete the user's request. Hand off tasks to the appropriate agent.",
    handoffs=[researcher_agent, writer_agent]
)

### Running the Agentic Workflow

In [11]:
# result = await 
await Runner.run(
    supervisor_agent,
    "Research the capabilities of the Gemini AI model and write a short summary to a file named 'gemini_summary.txt'.",
)

ImportError: cannot import name 'RunFinishedError' from 'anyio._core._exceptions' (/home/hydra/Desktop/GenAi/panaversity/OpenAIsdk/.venv/lib/python3.12/site-packages/anyio/_core/_exceptions.py)

In [ ]:
print(result.final_output)

Now, let's verify that the file was created.

In [ ]:
with open('gemini_summary.txt', 'r') as f:
    print(f.read())